In [159]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import string
import glob
from tqdm import tqdm
import random
import pickle
import string

In [160]:
# Get a list of all CSV files in the "Hist_Data" folder
df_meta = pd.read_csv('Metadata_CSV_MS.csv', delimiter = ';')

In [161]:
def get_alphabetic_id(n):
    alphabet = string.ascii_uppercase
    base = len(alphabet)
    result = ""

    while n > 0:
        n, remainder = divmod(n - 1, base)
        result = alphabet[remainder] + result

    return result

In [162]:
df_meta

,building,debtor_number,energy_usage,network,site,timeseries_name
0,4274,3774368,Endverbraucher,Dampf 30-35 bar,BUE,BUE2300009
1,4130,930792,Bezug Abhitze,Dampf 30-35 bar,BUE,BUE2300854
2,4130,930792,Bezug Abhitze,Dampf 30-35 bar,BUE,BUE2300855
3,4225,930792,Endverbraucher,Dampf 5-10 bar,BUE,BUE2600487
4,4106,930792,Bezug Abhitze,Dampf 5-10 bar,BUE,BUE2600851
...,...,...,...,...,...,...
4424,N31,930792,Endverbraucher,Strom,UER,UER4605140
4425,N259,4130170,Endverbraucher,Strom,UER,UER4605141
4426,L39,3190233,Endverbraucher,Strom,UER,UER4605142
4427,L39,3190233,Endverbraucher,Strom,UER,UER4605143


In [163]:
timeseries_mapping = {}

timeseries_name = df_meta["timeseries_name"]

for id,i in enumerate(timeseries_name):
    alphabetic_id = get_alphabetic_id(id+1)
    timeseries_mapping[alphabetic_id] = i

# Initialize an empty list to store DataFrames
dfs_list = []

for alphabetic_id, timeseries_name in timeseries_mapping.items():
    # Filter df_meta for rows with matching timeseries_name
    filtered_df = df_meta[df_meta['timeseries_name'] == timeseries_name]

    # Check if any matching rows were found
    if not filtered_df.empty:
        new_df = pd.DataFrame({
            'Dataset': [alphabetic_id],
            'meta_data_1': [filtered_df['building'].iloc[0]],
            'meta_data_2': [filtered_df['debtor_number'].iloc[0]],
            'meta_data_3': [filtered_df['energy_usage'].iloc[0]],
            'meta_data_4': [filtered_df['network'].iloc[0]],
            'meta_data_5': [filtered_df['site'].iloc[0]],
            'timeseriesname': [timeseries_name]
        })
        
        # Append new_df to the list
        dfs_list.append(new_df)

# Concatenate all the DataFrames in dfs_list
result_df = pd.concat(dfs_list, ignore_index=True)

# Display the resulting DataFrame
result_df
print(timeseries_mapping)

{'A': 'BUE2300009', 'B': 'BUE2300854', 'C': 'BUE2300855', 'D': 'BUE2600487', 'E': 'BUE2600851', 'F': 'BUE4600007', 'G': 'BUE4600008', 'H': 'BUE4600009', 'I': 'BUE4600010', 'J': 'BUE4600011', 'K': 'BUE4600012', 'L': 'BUE4600016', 'M': 'BUE4600020', 'N': 'BUE4600021', 'O': 'BUE4600479', 'P': 'BUE4600489', 'Q': 'BUE4600521', 'R': 'BUE4601175', 'S': 'BUE4602004', 'T': 'BUE4602005', 'U': 'BUE4602006', 'V': 'BUE4602007', 'W': 'BUE4605139', 'X': 'BUE4605140', 'Y': 'BUE4700577', 'Z': 'BUE4700601', 'AA': 'BUE4700639', 'AB': 'BUE4700659', 'AC': 'BUE4700661', 'AD': 'BUE4700662', 'AE': 'BUE4700666', 'AF': 'BUE4700674', 'AG': 'BUE4700677', 'AH': 'BUE4704659', 'AI': 'BUE4800001', 'AJ': 'BUE4800078', 'AK': 'BUE4800146', 'AL': 'BUE4800158', 'AM': 'BUE4800164', 'AN': 'BUE4800172', 'AO': 'BUE4800603', 'AP': 'BUE4800604', 'AQ': 'BUE4800890', 'AR': 'BUE4800891', 'AS': 'BUE4800892', 'AT': 'BUE4900113', 'AU': 'BUE4900270', 'AV': 'BUE4900271', 'AW': 'BUE4900272', 'AX': 'BUE4900276', 'AY': 'BUE4900278', 'AZ':

In [164]:
# Concatenate all the DataFrames in dfs_list
result_df = pd.concat(dfs_list, ignore_index=True)

# Define a random seed for reproducibility
random.seed(42)

# Define a mapping from original values to pseudonyms
pseudonym_mapping = {}

def generate_pseudonym(used_pseudonyms):
    # Generate a random pseudonym (in the format A:ZZZ) that hasn't been used yet
    while True:
        pseudonym = ''.join(random.choice(string.ascii_uppercase) for _ in range(3))
        if pseudonym not in used_pseudonyms:
            return pseudonym
        
# Extract unique combinations of 'Dataset' and 'timeseriesname'
unique_combinations = result_df[['Dataset', 'timeseriesname']].drop_duplicates()

# Pseudonymize meta_data columns (except meta_data_6)
for col in ['meta_data_1', 'meta_data_2', 'meta_data_3', 'meta_data_4', 'meta_data_5']:
    unique_values = result_df[col].unique()
    used_pseudonyms = set(pseudonym_mapping.values())
    for val in unique_values:
        if val not in pseudonym_mapping:
            pseudonym = generate_pseudonym(used_pseudonyms)
            pseudonym_mapping[val] = pseudonym
            used_pseudonyms.add(pseudonym)

# Apply pseudonyms to the DataFrame
for col in ['meta_data_1', 'meta_data_2', 'meta_data_3', 'meta_data_4', 'meta_data_5']:
    result_df[col] = result_df[col].map(pseudonym_mapping)

# Drop timeseriesname column (since it's already mapped to Dataset)
#result_df.drop(columns=['timeseriesname'], inplace=True)

#Save the Table
base_path = "Pseudonym_Table/"
file_path = f"{base_path}/Meta_data_Table.csv"
result_df.to_csv(file_path, sep=';', decimal=',', index=False)


# Save pseudonym_mapping
with open('Pseudonym_Table/pseudonym_mapping.pkl', 'wb') as f:
    pickle.dump(pseudonym_mapping, f)

# Save timeseries_mapping
with open('Pseudonym_Table/timeseries_mapping.pkl', 'wb') as f:
    pickle.dump(timeseries_mapping, f)

In [165]:
result_df

,Dataset,meta_data_1,meta_data_2,meta_data_3,meta_data_4,meta_data_5,timeseriesname
0,A,UDA,PUG,DTB,MZQ,BRN,BUE2300009
1,B,XIH,YJK,OTV,MZQ,BRN,BUE2300854
2,C,XIH,YJK,OTV,MZQ,BRN,BUE2300855
3,D,HEX,YJK,DTB,KWN,BRN,BUE2600487
4,E,DVX,YJK,OTV,KWN,BRN,BUE2600851
...,...,...,...,...,...,...,...
4424,FNE,YUG,YJK,DTB,NEJ,UCO,UER4605140
4425,FNF,SXY,JST,DTB,NEJ,UCO,UER4605141
4426,FNG,TNH,WFA,DTB,NEJ,UCO,UER4605142
4427,FNH,TNH,WFA,DTB,NEJ,UCO,UER4605143


In [166]:
#Check for Duplicates in other Columns
'''
# Print the Pseudonym Mapping
print("Pseudonym Mapping:")
for original, pseudonym in pseudonym_mapping.items():
    print(f"{original} -> {pseudonym}")

# Check for Duplicates in Pseudonym Columns
pseudonym_columns = ['meta_data_1', 'meta_data_2', 'meta_data_3', 'meta_data_4', 'meta_data_5']
for col in pseudonym_columns:
    duplicates = result_df[result_df.duplicated(col, keep=False)]
    if not duplicates.empty:
        print(f"Duplicate pseudonyms found in column '{col}':")
        print(duplicates)
'''

'\n# Print the Pseudonym Mapping\nprint("Pseudonym Mapping:")\nfor original, pseudonym in pseudonym_mapping.items():\n    print(f"{original} -> {pseudonym}")\n\n# Check for Duplicates in Pseudonym Columns\npseudonym_columns = [\'meta_data_1\', \'meta_data_2\', \'meta_data_3\', \'meta_data_4\', \'meta_data_5\']\nfor col in pseudonym_columns:\n    duplicates = result_df[result_df.duplicated(col, keep=False)]\n    if not duplicates.empty:\n        print(f"Duplicate pseudonyms found in column \'{col}\':")\n        print(duplicates)\n'